In [49]:
#Import dependencies
import pandas as pd
import pymongo
from bs4 import BeautifulSoup as bs
import requests
import os
from splinter import Browser
import re

In [50]:
#Locate chromedriver in local folder

!which chromedriver

/usr/local/bin/chromedriver


In [51]:
#Create executable path
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [52]:
#Scraping Nasa Mars website for latest news

url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [53]:
#Use bs to locate classes for news title and paragraph text

html = browser.html
soup = bs(html, 'html.parser')
#print(soup.prettify())

In [55]:
#Using find all function to identify latest title and text the print

news_title = soup.find('li', class_ = "slide").find('div', class_='content_title').text

title_text = soup.find('li', class_ = "slide").find('div', class_='article_teaser_body').text


print('-------------')
print(f"Title: {news_title}")
print(f"Synopsis: {title_text}")
print('-------------')      



-------------
Title: NASA's Perseverance Rover Is Midway to Mars 
Synopsis: Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.
-------------


In [39]:
#Scraping NASA JPL for featured Mars image

jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
jplbase = 'https://www.jpl.nasa.gov'
browser.visit(jpl_url)
browser.is_element_present_by_id("images", 1)

True

In [40]:
#Connect to website and find featured image url 
image_html = browser.html
image_bs = bs(image_html, 'html.parser')
feature_image = image_bs.find(id="images")
img_src = feature_image.find("img")["src"]
img_src

'/assets/images/logo_nasa_trio_black@2x.png'

In [42]:
featured_image_url = jplbase + img_src
featured_image_url

'https://www.jpl.nasa.gov/assets/images/logo_nasa_trio_black@2x.png'

In [43]:
#Scraping Space Facts for Mars data table
space_facts = 'https://space-facts.com/mars/'
browser.visit(space_facts)

In [44]:
#Connect to Space Facts website then convert findings to dataframe then again to html

mars_html = browser.html
mars_bs = bs(mars_html, 'html.parser')

mars_table = mars_bs.find('table', class_='tablepress tablepress-id-p-mars')

mars_df = pd.read_html(str(mars_table))[0]
mars_df.columns = ['Description','Parameters']
mars_df = mars_df.set_index(['Description'])

mars_html = mars_df.to_html().replace("\n", "")
mars_html

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Parameters</th>    </tr>    <tr>      <th>Description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [45]:
#Connect to Mars hemisphere website 
mars_hemisphere = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemisphere)
    

In [46]:
#Locate hemisphere images then create dictionary to contain these images and their titles
mars_hemisphere_html = browser.html
mars_hemisphere_bs = bs(mars_hemisphere_html, 'html.parser')
mars_hemisphere = mars_hemisphere_bs.find_all('img', class_ ='thumb')

mars_hemisphere_image_urls= []

for hemisphere in mars_hemisphere:
    image_title = hemisphere.attrs['alt'].replace('Enhanced thumbnail','')
    image_url = hemisphere.attrs['src']
    mars_hemisphere_image_urls.append({'Title':image_title,'Image URl':image_url})

mars_hemisphere_image_urls


[{'Title': 'Cerberus Hemisphere ',
  'Image URl': '/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'},
 {'Title': 'Schiaparelli Hemisphere ',
  'Image URl': '/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'},
 {'Title': 'Syrtis Major Hemisphere ',
  'Image URl': '/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'},
 {'Title': 'Valles Marineris Hemisphere ',
  'Image URl': '/cache/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]

In [47]:
browser.back()

In [48]:
browser.quit()